In [18]:
import pandas as pd
import tensorflow as tf
import preprocess_kgptalkie as ps

Importing Data + Filtering
* total data count: 7978

In [201]:
# training data
train_dataset = pd.read_csv('drugsComTrain_raw.csv')
train_dataset.dropna(inplace=True)

antidepressants_list = ['Escitalopram', 'Vilazodone', 'Lexapro', 'Trintellix', 'Paroxetine', 'Sertraline', 
                        'Citalopram', 'Fluoxetine', 'Zoloft', 'Celexa', 'Prozac', 'Venlafaxine', 'Pristiq', 
                        'Desvenlafaxine', 'Duloxetine', 'Cymbalta', 'Viibryd', 'Effexor', 'Effexor XR', 
                        'Amitriptyline', 'Amoxapine', 'Desipramine', 'Doxepin', 'Imipramine', 'Nortriptyline', 
                        'Protriptyline', 'Trimipramine', 'Isocarboxazid', 'Phenelzine', 'Selegiline', 
                        'Tranylcypromine']

train_dataset = train_dataset[train_dataset.drugName.isin(antidepressants_list)]
train_dataset = train_dataset[train_dataset.condition.isin(['Depression'])]

test_dataset = pd.read_csv('drugsComTest_raw.csv')
test_dataset.dropna(inplace=True)
test_dataset = test_dataset[test_dataset.condition.isin(['Depression'])]
test_dataset = test_dataset[test_dataset.drugName.isin(antidepressants_list)]

In [202]:
frames = [train_dataset, test_dataset]
dataset = pd.concat(frames)

In [203]:
dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
31,96233,Sertraline,Depression,"""1 week on Zoloft for anxiety and mood swings....",8,7-May-11,3
44,121333,Venlafaxine,Depression,"""my gp started me on Venlafaxine yesterday to ...",4,27-Apr-16,3
67,131909,Effexor XR,Depression,"""This medicine saved my life. I was at my wits...",10,20-Jun-13,166
143,186257,Desvenlafaxine,Depression,"""This medication is amazing! After 3 days of b...",10,10-Mar-13,101
150,45238,Fluoxetine,Depression,"""My genius psychiatrist started me on this dru...",1,11-Jan-16,19
...,...,...,...,...,...,...,...
53580,96067,Sertraline,Depression,"""I have been taking this medicine for years af...",10,23-Jul-12,10
53582,205800,Pristiq,Depression,"""I have been on Pristiq for almost 2 weeks. I...",10,28-Feb-11,15
53600,28837,Lexapro,Depression,"""Lexapro quickly lifted me out of depression a...",8,27-Oct-09,12
53691,61639,Citalopram,Depression,"""Best medication ever I&#039;m on 20 mg .its v...",10,22-Jul-16,43


Preprocessing
* lowercase, punctuation removal, stopword removal

In [204]:
import gensim

In [205]:
dataset['review'] = dataset['review'].apply(lambda x: str(x).lower())
dataset['review'] = dataset['review'].apply(gensim.parsing.preprocessing.strip_punctuation)

In [206]:
from gensim.parsing import remove_stopwords
dataset['review'] = dataset['review'].apply(gensim.parsing.preprocessing.remove_stopwords)

In [207]:
dataset

,uniqueID,drugName,condition,review,rating,date,usefulCount
31,96233,Sertraline,Depression,1 week zoloft anxiety mood swings 50mg morning...,8,7-May-11,3
44,121333,Venlafaxine,Depression,gp started venlafaxine yesterday help depressi...,4,27-Apr-16,3
67,131909,Effexor XR,Depression,medicine saved life wits end anti depressants ...,10,20-Jun-13,166
143,186257,Desvenlafaxine,Depression,medication amazing 3 days extremely sick start...,10,10-Mar-13,101
150,45238,Fluoxetine,Depression,genius psychiatrist started drug fall 2008 wee...,1,11-Jan-16,19
...,...,...,...,...,...,...,...
53580,96067,Sertraline,Depression,taking medicine years spending years trying me...,10,23-Jul-12,10
53582,205800,Pristiq,Depression,pristiq 2 weeks medicine past 10 yrs started z...,10,28-Feb-11,15
53600,28837,Lexapro,Depression,lexapro quickly lifted depression kept way imp...,8,27-Oct-09,12
53691,61639,Citalopram,Depression,best medication 039 m 20 mg important medicati...,10,22-Jul-16,43


In [209]:
dataset['tokens'] = dataset.apply(lambda row: nltk.word_tokenize(row['review']), axis=1)

In [247]:
sentences = dataset['tokens'].tolist()

Word2Vec

In [279]:
from gensim.models import Word2Vec
w2v = Word2Vec(window=4, workers=4)
w2v.build_vocab(dataset.tokens)
w2v.train(dataset.tokens, total_examples=w2v.corpus_count, epochs = w2v.epochs)
w2v.save("word2vec.model")

In [280]:
word_vectors = w2v.wv
word_vectors.save("word2vec.wordvectors")

# Load back with memory-mapping = read-only, shared across processes.
wv = KeyedVectors.load("word2vec.wordvectors", mmap='r')

In [289]:
import numpy as np
def sent_vec(sent):
    vector_size = wv.vector_size
    wv_res = np.zeros(vector_size)
    # print(wv_res)
    ctr = 1
    for w in sent:
        if w in wv:
            ctr += 1
            wv_res += wv[w]
    wv_res = wv_res/ctr
    return wv_res

In [290]:
dataset['vec'] = dataset['tokens'].apply(sent_vec)

In [298]:
X = dataset['vec'].to_list()
y = dataset['rating'].to_list()

In [303]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

from sklearn.tree import DecisionTreeRegressor
model = DecisionTreeRegressor()

model.fit(X_train,y_train)

DecisionTreeRegressor()

In [306]:
from sklearn import metrics
predicted = model.predict(X_test)
print(metrics.accuracy_score(y_test, predicted))
print(metrics.precision_score(y_test, predicted, average = 'micro'))
print(metrics.precision_score(y_test, predicted, average = 'macro'))
print(metrics.precision_score(y_test, predicted, average = 'weighted'))
print(metrics.recall_score(y_test, predicted, average = 'micro'))
print(metrics.recall_score(y_test, predicted, average = 'macro'))
print(metrics.recall_score(y_test, predicted, average = 'weighted'))

0.6972431077694236
0.6972431077694236
0.6696161644950398
0.6988364621420291
0.6972431077694236
0.6622150049242862
0.6972431077694236
